**Student Information**

Môn học: Toán Ứng dụng và Thống kê

Họ tên: Bế Lã Anh Thư

Lớp: 22CLC02

MSSV: 22127402

# Trình bày giải thuật chéo hóa ma trận
## <center> __Thuật toán chéo hóa ma trận vuông  A__ </center>
Với ma trận vuông $A \in M_n$, ma trận chéo hóa được nếu tồn tại ma trận khả nghịch $P$ sao cho $D = P^{-1}AP$ với $D$ là một ma trận chéo. Khi đó ta nói $P$ làm chéo hóa $A$ và $D$ là dạng chéo của $A$. Điều này có thể thực hiện được nếu $A$ có đầy đủ các trị riêng và vector riêng tương ứng.

- __Bước 1__: Tìm đa thức đặc trưng $P_A(\lambda) = det(A - \lambda I)$. Nếu $P_A(\lambda)$ có tổng các lũy thừa khác $n$ thì $A$ không chéo hóa được, thuật toán kết thúc, ngược lại chuyển sang bước 2.
  
- __Bước 2__: Tìm tất cả các nghiệm $\lambda_i$ của phương trình đa thức đặc trưng. Với mỗi trị riêng $\lambda_i$, tìm cơ sở và số chiều cho không gian nghiệm mỗi phương trình $(A - \lambda I_n)X = 0$. Nếu mỗi $\lambda_i$ số chiều không gian nghiệm nhỏ hơn lũy thừa của $\lambda_i$ trong đa thức đặc trưng thì $A$ không chéo hóa được, thuật toán kết thúc, ngược lại chuyển sang bước 3.
  
- __Bước 3__: Với các vector trong cơ sở không gian nghiệm tìm được ở bước 2, ta đặt ma trận $P$ là ma trận cột (có được bằng cách dựng các vector thành các cột). Khi đó, $P$ làm chéo $A$ và $D = P^{-1}AP$ là ma trận đường chéo. 
$$ diag(\lambda_1, \dots, \lambda_r) $$

# Chéo hóa ma trận

In [10]:
from fractions import Fraction
import sympy as sp
import numpy as np

def swap_col(A, pre_index, new_index):
    ''' 
        swap 2 column of the matrix
    '''
    A[pre_index], A[new_index] = A[new_index], A[pre_index]

def create_identity_matrix(n):
    ''' 
        create the identity matrix size nxn
    '''
    return [[1 if row == col else 0 for col in range(n)] for row in range(n)]

def print_matrix(A):
    '''
        print matrix
    '''
    for row in A:
        for element in row:
            try:
                element = float(element)
            except (ValueError, TypeError):
                pass
            print(Fraction(element).limit_denominator(1000), end=' ')
        print()

def gauss_jordan(A, I):
    """
        Generate the inversion matrix of A.
        Input: A - square matrix, size nxn, I - identity matrix, size nxn.
        Output: Inversion matrix of A, size nxn
    """
    n = len(A)
    m = len(A[0])
    pivot = {
        "row": 0,
        "col": 0
    }

    if (m!=n):
        # if matrix is not square
        return []

    while pivot["row"] < n and pivot["col"] < m:
        
        if A[pivot["row"]][pivot["col"]] == 0:
            change_pivot = False

            for row in range(pivot["row"] + 1, n):
                # if there is cell with value != 0 in same col with pivot, swap that row with pivot row.
                if A[row][pivot["col"]] != 0:
                    swap_col(A, pivot["row"], row)
                    swap_col(I, pivot["row"], row)
                    change_pivot = True
                    break
            
            if not change_pivot:
                # no non-zero element found in same column w pivot, move to the next coiumn
                return []

        # eliminate to get leading 1
        leading = Fraction(1/A[pivot["row"]][pivot["col"]])
        for ele in range(m):
            A[pivot["row"]][ele] *= leading
            I[pivot["row"]][ele] *= leading
        
        # eliminate other col to zero
        for ele_row in range(n):
            if ele_row == pivot["row"]:
                continue
            coefficient = A[ele_row][pivot["col"]]
            for ele_col in range(m):
                if ele_col >= pivot["row"]:
                    A[ele_row][ele_col] -= coefficient*A[pivot["row"]][ele_col]
                I[ele_row][ele_col] -= coefficient*I[pivot["row"]][ele_col]
        
        pivot["row"] += 1
        pivot["col"] += 1
    
    return I

def inversion(A):
    '''
        create the inversion matrix of matrix A
    '''
    I = create_identity_matrix(len(A))
    invertible_matrix = gauss_jordan(A, I)
    
    return invertible_matrix

def multi_matrix(A, B):
    '''
        multiply two matrix
    '''
    result = [[0 for col in range(len(B[0]))] for row in range(len(A))]
    for i in range(len(A)):
        for j in range(len(B[0])):
            for k in range(len(B)):
                result[i][j] += A[i][k] * B[k][j]
    return result

def determinant(A):
    '''
        calculate the determinant of matrix A
    '''
    return np.linalg.det(A)

def get_eigenvalues(A):
    '''
        find all the eigenvalues of matrix A
        return list of eigen
    '''
    x = sp.symbols('x') 
    char_poly = A.charpoly(x).as_expr()
    eigenvalues = sp.solve(char_poly, x)
    return [val.evalf() for val in eigenvalues if sp.im(val) == 0]

def get_eigenvectors(A, eigenvalues):
    ''' 
        create the eigenvector
    '''
    eigenvectors = []
    for val in eigenvalues:
        A_val = A - val * sp.eye(A.shape[0])
        null_space = A_val.nullspace()
        eigenvectors.extend(null_space)
    return eigenvectors

def diagonalize_matrix(A):
    """
    Diagonalize matrix A 
    Input: A - 2D list representing a square matrix.
    Output: P - matrix of eigenvectors, D - diagonal matrix of eigenvalues, P_inv - inverse of P.
    """
    sympy_matrix = sp.Matrix(A)
    eigenvalues = get_eigenvalues(sympy_matrix)

    if not eigenvalues or len(eigenvalues) != sympy_matrix.shape[0]:
        print("Không thể chéo hóa ma trận")
        return None, None, None

    eigenvectors = get_eigenvectors(sympy_matrix, eigenvalues)

    if len(eigenvectors) != sympy_matrix.shape[0]:
        print("Không thể chéo hóa ma trận")
        return None, None, None

    P = sp.Matrix.hstack(*eigenvectors)
    P_inv = sp.Matrix(inversion(np.array(P).astype(float).tolist()))

    D = P_inv * sympy_matrix * P

    return np.array(P).astype(float), np.array(D).astype(float), np.array(P_inv).astype(float)

# Kiểm tra kết quả

In [11]:
bt1a = [
    [-1, 3],
    [-2, 4]
]
bt1b = [
    [5, 2],
    [9, 2]
]
bt1c = [
    [1, -1, -1],
    [1, 3, 1],
    [-3, 1, -1]
]
bt1d = [
    [5, -1, 1],
    [-1, 2, -2],
    [1, -2, 2]
]
bt1e = [
    [1, 3, 3],
    [-3, -5, -3],
    [3, 3, 1]
]
bt1f = [
    [4, 0, -1],
    [0, 3, 0],
    [1, 0, 2]
]
bt1g = [
    [3, 4, -4],
    [-2, -1, 2],
    [-2, 0, 1]
]
bt1h = [
    [0, 0, -2],
    [1, 2, 1],
    [1, 0, 3]
]
bt1i = [
    [1, 0, 0],
    [1, 2, 0],
    [-3, 5, 2]
]
bt1j = [
    [4, 0, 1],
    [-2, 1, 0],
    [-2, 0, 1]
]

bt1 = [bt1a, bt1b, bt1c, bt1d, bt1e, bt1f, bt1g, bt1h, bt1i, bt1j]

for A in bt1:
    print(f'\n================================')
    print("A:")
    print_matrix(A)

    P, D, P1 = diagonalize_matrix(A)
    if P is not None:
        print("Ma tran P:")
        print_matrix(P)
        print("Ma tran D:")
        print_matrix(D)
        print("Ma tran P^(-1):")
        print_matrix(P1)

        B = multi_matrix(multi_matrix(P, D), P1)
        print("P*D*P^(-1):")
        print_matrix(B)


A:
-1 3 
-2 4 
Ma tran P:
3/2 1 
1 1 
Ma tran D:
1 0 
0 2 
Ma tran P^(-1):
2 -2 
-2 3 
P*D*P^(-1):
-1 3 
-2 4 

A:
5 2 
9 2 
Ma tran P:
-1/3 2/3 
1 1 
Ma tran D:
-1 0 
0 8 
Ma tran P^(-1):
-1 2/3 
1 1/3 
P*D*P^(-1):
5 2 
9 2 

A:
1 -1 -1 
1 3 1 
-3 1 -1 
Ma tran P:
1/4 -1 -1 
-1/4 0 1 
1 1 1 
Ma tran D:
-2 0 0 
0 2 0 
0 0 3 
Ma tran P^(-1):
4/5 0 4/5 
-1 -1 0 
1/5 1 1/5 
P*D*P^(-1):
1 -1 -1 
1 3 1 
-3 1 -1 

A:
5 -1 1 
-1 2 -2 
1 -2 2 
Ma tran P:
0 -1 2 
1 -1 -1 
1 1 1 
Ma tran D:
0 0 0 
0 3 0 
0 0 6 
Ma tran P^(-1):
0 1/2 1/2 
-1/3 -1/3 1/3 
1/3 -1/6 1/6 
P*D*P^(-1):
5 -1 1 
-1 2 -2 
1 -2 2 

A:
1 3 3 
-3 -5 -3 
3 3 1 
Không thể chéo hóa ma trận

A:
4 0 -1 
0 3 0 
1 0 2 
Không thể chéo hóa ma trận

A:
3 4 -4 
-2 -1 2 
-2 0 1 
Ma tran P:
1 0 -1 
0 1 1 
1 1 1 
Ma tran D:
-1 0 0 
0 1 0 
0 0 3 
Ma tran P^(-1):
0 -1 1 
1 2 -1 
-1 -1 1 
P*D*P^(-1):
3 4 -4 
-2 -1 2 
-2 0 1 

A:
0 0 -2 
1 2 1 
1 0 3 
Không thể chéo hóa ma trận

A:
1 0 0 
1 2 0 
-3 5 2 
Không thể chéo hóa ma trận

A:
4 0 1 
-

# Ứng dụng của chéo hóa
##  <center> __Phân tích các hệ thống động lực__ </center>
Một trong những ứng dụng quan trọng của chéo hóa ma trận là trong phân tích các hệ thống động lực tuyến tính. Ví dụ, xem xét một hệ thống động lực tuyến tính được mô phỏng bởi phương trình vi phân ma trận sau:

$$ \frac{dx}{dt} = Ax $$
trong đó, $x$ là vector trạng thái, $A$ là một ma trận hệ số. Ta có thể giải quyết bài toán này dễ dàng hơn thông qua việc chéo hóa ma trận $A$.

Khi ma trận $A$ được chéo hóa, phương trình vi phân trở nên dễ giải hơn vì ma trận chéo $D$ có các trị riêng của $A$ trên đường chéo chính ($diag(\lambda_1, \dots, \lambda_r)$), và hệ phương trình lúc này sẽ trở nên tách rời, tức mỗi phương trình riêng rẽ có thể giải quyết một cách độc lập.

## Chương trình minh họa

In [12]:
def solve_diferential_sys(A, x, t):
    '''
        Solve dx/dt = Ax
        A: ma tran ban dau
        x: vector ma tran ban dau
        t: thoi gian (mang)
    '''
    P, D, P1 = diagonalize_matrix(A)
    if P is None or D is None or P1 is None:
        return None
    
    # bieu dien nghiem tong quat
    C = np.dot(P1, x)
    X_t = np.zeros((len(A), len(t)))

    for i, ti in enumerate(t):
        X_t[:,i] = np.dot(P, np.dot(np.diag(np.exp(np.diag(D) * ti)), C)).flatten()

    
    return X_t

# example
x = np.array([1, 0])
t = np.linspace(0, 5)

X_t = solve_diferential_sys(bt1a, x, t)

if X_t is not None:
    for i, xt in enumerate(X_t):
        print(f"x{i}{t} = {xt}")

x0[0.         0.10204082 0.20408163 0.30612245 0.40816327 0.51020408
 0.6122449  0.71428571 0.81632653 0.91836735 1.02040816 1.12244898
 1.2244898  1.32653061 1.42857143 1.53061224 1.63265306 1.73469388
 1.83673469 1.93877551 2.04081633 2.14285714 2.24489796 2.34693878
 2.44897959 2.55102041 2.65306122 2.75510204 2.85714286 2.95918367
 3.06122449 3.16326531 3.26530612 3.36734694 3.46938776 3.57142857
 3.67346939 3.7755102  3.87755102 3.97959184 4.08163265 4.18367347
 4.28571429 4.3877551  4.48979592 4.59183673 4.69387755 4.79591837
 4.89795918 5.        ] = [ 1.00000000e+00  8.69489489e-01  6.71089389e-01  3.85310560e-01
 -1.21909510e-02 -5.51760568e-01 -1.27115655e+00 -2.21728656e+00
 -3.44834360e+00 -5.03643254e+00 -7.07079889e+00 -9.66179710e+00
 -1.29457667e+01 -1.70910228e+01 -2.23052145e+01 -2.88443616e+01
 -3.70239523e+01 -4.72325678e+01 -5.99486099e+01 -7.57608347e+01
 -9.53935561e+01 -1.19737579e+02 -1.49888162e+02 -1.87191606e+02
 -2.33302413e+02 -2.90253439e+02 -3.60541947e+